In [ ]:
import numpy as np
import pandas as pd
import h5py

def read_file(file_name):
	file = open(file_name)
	X = []
	k = 0
	while 1:
		line = file.readline()
		k = k+1
		if k==420000:
			break
		if not line:
			break
		data = line[:-1].split(',')
		X.append(np.array(data).astype(np.float64))
	X = np.array(X)
	return X

def coef_wave(coef):
    a = []
    for i in range(1,1017):
        kappa=0
        for j in range(1,7):
            kappa = kappa+(coef[j-1])*(i**(j-1))
        a.append(kappa)
    return a

spec_files = [
    'spec_16.dat',
    'spec_17.dat',
    'spec_18.dat',
    'spec_19.dat',
    'spec_20.dat'
]

train_files = [
    'train_16.dat',
    'train_17.dat',
    'train_18.dat',
    'train_19.dat',
    'train_20.dat'
]

save_files = [
    'exp_16.h5',
    'exp_17.h5',
    'exp_18.h5',
    'exp_19.h5',
    'exp_20.h5'
]

def delta_sco2(x):
    return x*(-0.00034774683749767404)+0.00075751732302189
def delta_wco2(x):
    return x*(-0.00036041255295181477)+0.0006099674364339724
n_sco2 = 755
n_wco2 = 525
x_sco2 = []
x_wco2 = []
x_sco2.append(2.0478)
for i in range(n_sco2-1):
    x_sco2.append(x_sco2[i]+delta_sco2(x_sco2[i]))

x_wco2.append(1.599)
for i in range(n_wco2-1):
    x_wco2.append(x_wco2[i]+delta_wco2(x_wco2[i]))

def spec_raw_intp(vecter,wco2,sco2):
    spec_inter = []
    bad_level = []
    for i in range(len(vecter)):
        wco2_coef = coef_wave(vecter[i,4066:4072])
        sco2_coef = coef_wave(vecter[i,4072:4078])
        a = np.append(np.interp(wco2,wco2_coef,vecter[i,2:1018]),np.interp(sco2,sco2_coef,vecter[i,1018:2034]),axis=0)
        b = np.append(np.interp(wco2,wco2_coef,vecter[i,2034:3050]),np.interp(sco2,sco2_coef,vecter[i,3050:4066]),axis=0)
        spec_inter.append(a)
        bad_level.append(b)
        print(str(i)+'/'+str(len(vecter)))
    return np.array(spec_inter), np.array(bad_level)

In [ ]:
for spec_file, train_file,save_file in zip(spec_files,train_files,save_files):
    X = np.asarray(read_file(spec_file))
    y = np.asarray(read_file(train_file))
    spec_inter_1, bad_level_inter = spec_raw_intp(X,x_wco2,x_sco2)
    for i in range(len(spec_inter_1)):
        spec_inter_1[i, 0:525] = spec_inter_1[i, 0:525] / np.mean(spec_inter_1[i, 206:216])
        spec_inter_1[i, 525:1280] = spec_inter_1[i, 525:1280] / np.mean(spec_inter_1[i, 811:826])

    with h5py.File(save_file, 'w') as hf:
        hf.create_dataset('x', data=spec_inter_1)
        hf.create_dataset('bad_level', data=bad_level_inter)
        hf.create_dataset('y', data=y)
    